# Getting Started

Go to the setupTable and input your unique token

DO NOT USE MY KEY (Blake)

In [1]:
import requests
import time

In [2]:
class Stack():
    def __init__(self):
        self.stack = []
    def push(self, value):
        self.stack.append(value)
    def pop(self):
        if self.size() > 0:
            return self.stack.pop()
        else:
            return None
    def size(self):
        return len(self.stack)

In [213]:
class Queue():
    def __init__(self):
        self.queue = []
    def enqueue(self, value):
        self.queue.append(value)
    def dequeue(self):
        if self.size() > 0:
            return self.queue.pop(0)
        else:
            return None
    def size(self):
        return len(self.queue)

In [210]:
# Import map and room info
with open('worldMap.json', 'r') as f:
    worldMap = json.load(f)
    
with open('worldRoomIndex.json', 'r') as f:
    worldRoomIndex = json.load(f)

In [4]:
# Utils
inverseMap = {'n':'s', 's':'n', 'e':'w', 'w':'e'}

In [6]:
setupTable = {
    "url": 'https://lambda-treasure-hunt.herokuapp.com/api/',
    "token": "d2679616e63cea70c89ff019addc2677a27c9388",
    "init": 'adv/init/',
    "move": 'adv/move/',
    "take": 'adv/take/'
}
header={'Authorization': 'Token ' + setupTable['token']}
init = setupTable['url'] + setupTable['init']
move = setupTable['url'] + setupTable['move']
take = setupTable['url'] + setupTable['take']

In [351]:
class Player:
    
    def __init__(self):
        self.initResponse = requests.get(url=init, headers=header)
        self.playersMap = worldMap
        self.coolDown = None
        self.currentRoomID = None
        self.previousRoomID = None
        self.lastMoveDirection = None
        self.playerItems = []
        
        self.currentRoomInfo = None
        
    def getRoomInfo(self):
        time.sleep(2)
        response = requests.get(url=init, headers=header)
        
        if response.status_code == 200:
            return response.json()
        else:
            print(response.json())
            print(f'400 status code at getRoomInfo method {response.status_code}')
            
    def assignCurrentRoomInfo(self):
        self.currentRoomInfo = self.getRoomInfo()
        
    def assignCurrentRoomID(self):
        self.currentRoomID = self.getCurrentRoomID()
            
    def getCurrentRoomID(self):
        return self.currentRoomInfo['room_id']
            
    def writeNewRoomToMap(self):
        exits = self.currentRoomInfo['exits']
        
        if self.currentRoomID not in self.playersMap:
            self.playersMap[self.currentRoomID] = {key: '?' for key in exits}
            
    def writeMoveDirectionsForCurrentAndPreviousRoom(self, previousRoomID, currentRoomID, direction):
        self.playersMap[previousRoomID][direction] = self.currentRoomID
        self.playersMap[currentRoomID][inverseMap[direction]] = self.previousRoomID
        
    def moveRoom(self, direction):
        if self.currentRoomID in self.playersMap and self.playersMap[self.currentRoomID][direction] !='?':
            print("Wise Explore")
            
            self.previousRoomID = self.currentRoomID
            
            nextRoom = self.playersMap[self.currentRoomID][direction]
            
            postRequestBody = {"direction":direction, "next_room_id":str(nextRoom)}
            response = requests.post(url=move, headers=header, json=postRequestBody)
            self.currentRoomInfo = response.json()
            self.coolDown = response.json()['cooldown']
            
            time.sleep(self.coolDown)
            self.currentRoomID = self.getCurrentRoomID()
            self.lastMoveDirection = direction
            
        else:
            print('Normal Explore')
            self.previousRoomID = self.currentRoomID
            
            postRequestBody = {"direction":direction}
            response = requests.post(url=move, headers=header, json=postRequestBody)

            self.currentRoomInfo = response.json()
            self.currentRoomID = self.getCurrentRoomID()
            
            if response.status_code == 200:
                self.coolDown = response.json()['cooldown']
                time.sleep(self.coolDown)
                
                self.lastMoveDirection = direction
                
                self.writeNewRoomToMap()
                self.writeMoveDirectionsForCurrentAndPreviousRoom(self.previousRoomID, self.currentRoomID, direction)
            else:
                print("The move was not a success")
                
        return self.getRoomInfo()
    
    def findUnexploredRoom(self):
        for direction in self.playersMap[self.currentRoomID]:
            if self.playersMap[self.currentRoomID][direction] == '?':
                return direction
        return direction
    
    def takeItem(self, itemName):
        postRequestBody = {'name': itemName}
        response = requests.post(url=move, headers=header, json=postRequestBody)
        self.coolDown = response.json()['cooldown']
        
        self.playerItems.append(itemName)
        print("You have picked up ", itemName)
        
    def dropItem(self, itemName):
        pass
        
    def checkInventory(self):
        print(self.playerItems)
    

In [352]:
def exploreWorld(player):
    print('Initializing...')
    
    roomStack = Stack()
    moveStack = Stack()
    count = 0
    

    player.currentRoomInfo = player.getRoomInfo()
    print('Loading currentRoomInfo')
    time.sleep(5)
    
    player.currentRoomID = player.currentRoomInfo['room_id']
    player.writeNewRoomToMap()
    
    roomStack.push(player.currentRoomID)
    
    roomDict = {}
    roomDict[player.currentRoomID] = player.currentRoomInfo
    
    while roomStack.size() > 0:
        
        direction = player.findUnexploredRoom()
        print("Direction", direction)
        
        if player.playersMap[player.currentRoomID][direction] == '?':
            
            
            player.moveRoom(direction)
            moveStack.push(direction)
            
            print("Leaving room: "+str(player.previousRoomID)+" entering room: "+str(player.currentRoomID))
            
            roomStack.push(player.currentRoomID)
            
            if player.currentRoomID not in roomDict:
                roomDict[player.currentRoomID] = player.currentRoomInfo
                
        else:
            print("roomStack:", roomStack.stack)
            print('moveStack:', moveStack.stack)
            roomStack.pop()
            lastMove = moveStack.stack[-1]
            backTrackingDirection = inverseMap[lastMove]
            
            if roomStack.size() > 0:
                player.previousRoomID = player.currentRoomID
                player.moveRoom(backTrackingDirection)
                player.lastMoveDirection = moveStack.stack[-1]      

            moveStack.pop()

    return roomDict          

In [362]:
def bfs(player, starting_vertex, destination_vertex):
    """
    Return a list containing the shortest path from
    starting_vertex to destination_vertex in
    breath-first order.
    """

    # Create an empty queue and enqueue the starting vertex ID
    pathQueue = Queue()
    directionQueue = Queue()
    
    pathQueue.enqueue([str(starting_vertex)])
    directionQueue.enqueue(['Starting Location'])

    # Create a Set to store visited vertices
    visited = set()

    # While the queue is not empty...
    while destination_vertex not in visited:
        # Dequeue the first vertex
        path = pathQueue.dequeue()
        directionPath = directionQueue.dequeue()

        # If that vertex has not been visited
        v = path[-1]
        
        if v not in visited:
            if v == destination_vertex:
                return path, directionPath
            visited.add(v)
            
        for direction, neighboringRoom in player.playersMap[v].items():
            path_copy = list(path)
            path_copy.append(str(neighboringRoom))
            pathQueue.enqueue(path_copy)
            
            directionCopy = list(directionPath)
            directionCopy.append(direction)
            directionQueue.enqueue(directionCopy)

    return None

In [382]:
def dfs(player, starting_vertex, destination_vertex):
    """
    Return a list containing the shortest path from
    starting_vertex to destination_vertex in
    breath-first order.
    """

    # Create an empty queue and enqueue the starting vertex ID
    pathStack = Stack()
    directionStack = Stack()
    
    pathStack.push([str(starting_vertex)])
    directionStack.push(['Starting Location'])

    # Create a Set to store visited vertices
    visited = set()

    # While the queue is not empty...
    while destination_vertex not in visited:
        # Dequeue the first vertex
        path = pathStack.pop()
        directionPath = directionStack.pop()

        # If that vertex has not been visited
        v = path[-1]
        
        if v not in visited:
            if v == destination_vertex:
                return path, directionPath
            visited.add(v)
            
        for direction, neighboringRoom in player.playersMap[v].items():
            path_copy = list(path)
            path_copy.append(str(neighboringRoom))
            pathStack.push(path_copy)
            
            directionCopy = list(directionPath)
            directionCopy.append(direction)
            directionStack.push(directionCopy)

    return None

In [370]:
def moveToSpecificRoom(player, starting_vertex, destination_vertex):
    path, directionPath = bfs(player, starting_vertex, destination_vertex)
    
    print(path)
    directions = directionPath[1:]
    print(directions)
    for direction in directions:
        player.moveRoom(direction)
        
        print("You have reached your destination")

In [375]:
player = Player()
player.assignCurrentRoomInfo()
player.assignCurrentRoomID()

In [376]:
player.currentRoomID

362

In [384]:
player.getRoomInfo()

{'room_id': 362,
 'title': 'A misty room',
 'description': 'You are standing on grass and surrounded by a dense mist. You can barely make out the exits in any direction.',
 'coordinates': '(68,49)',
 'elevation': 0,
 'terrain': 'NORMAL',
 'players': ['player464'],
 'items': [],
 'exits': ['n', 's', 'w'],
 'cooldown': 1.0,
 'errors': [],
 'messages': []}